# Sentence Similarity

In [4]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 11.9 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 18.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [optimum-onnx] [optimum]
Note: you may need to restart the kernel to use updated packages.


In [37]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from datasets import load_dataset, Dataset
import torch
from huggingface_hub import login

In [7]:
login()

In [21]:
model=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2",
                          model_kwargs={"dtype": torch.bfloat16}).to("cuda")
ds=load_dataset("harsha2946/ParaDetox-Augmented")

README.md:   0%|          | 0.00/383 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/39488 [00:00<?, ? examples/s]

In [22]:
def add_similarity_column(ex):
    embeddings=model.encode([ex["en_toxic_comment"], ex["en_neutral_comment"]])
    similarity=model.similarity(embeddings, embeddings)
    return {
        "sentence_similarity": similarity[0][1].item()
    }

In [23]:
ds=ds.map(add_similarity_column, batched=False)

Map:   0%|          | 0/39488 [00:00<?, ? examples/s]

In [31]:
df=ds.to_pandas()

In [32]:
df=df.sort_values(by="sentence_similarity", ascending=False)
df.describe()

,sentence_similarity
count,39488.000000
mean,0.737618
std,0.191921
min,-0.002673
25%,0.617886
50%,0.781865
75%,0.893694
max,1.000000


In [14]:
ds=Dataset(df, preserve_index=False)
ds.push_to_hub("harsha2946/ParaDetox-Augmented")

,en_toxic_comment,en_neutral_comment,sentence-similarity
0,he had steel balls too !,he was brave too!,0.448762
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...,0.673561
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to...",0.906688
3,the garbage that is being created by cnn and o...,the news that is being created by cnn and othe...,0.832897
4,the reason they dont exist is because neither ...,The reason they don't exist is because neither...,0.955393
...,...,...,...
39483,When they engage in such behavior.,When they engage in actions like this,0.833991
39484,"However, if expressing ""fuck that group"" is mo...","But if you find the phrase ""that group is bad""...",0.778219
39485,It's disheartening to see how judgmental jerks...,It's disheartening to see how people judge the...,0.825192
39486,"Damn, we'll probably demolish that in just a w...",We most likely obliterate that in a week.,0.674461


In [35]:
ds[0]

{'en_toxic_comment': 'he had steel balls too !',
 'en_neutral_comment': 'he was brave too!',
 'sentence_similarity': 0.44876179099082947}

In [34]:
df

,en_toxic_comment,en_neutral_comment,sentence_similarity
29675,"I'm not here to spread negativity, but it seem...","I'm not here to spread negativity, but it seem...",1.000000
33772,Why do you care?,Why do you care?,1.000000
33913,Make sure to prioritize your own family's needs.,Make sure to prioritize your own family's needs.,1.000000
1227,this is the type of ignorant response to which...,this is the type of ignorant response to which...,1.000000
3705,freeland acted like a petulant little kid .,freeland acted like a petulant little kid .,1.000000
...,...,...,...
27083,"Thanks a lot, Gab > You're such a moron for ag...",She is in concordance with me,0.010379
22484,Real supporters like yourself are simply ideal...,Correct,0.007445
29864,"I'm in a really bad situation, things are pret...","I don't possess one, I'm finished for!",0.005237
20257,"Indeed, the actual culprit is the fool who lea...",Genuine,0.003418


In [36]:
ds.push_to_hub("harsha2946/ParaDetox-Augmented")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/harsha2946/ParaDetox-Augmented/commit/a8dd8ffe5d1d061882bb7bd6b971c0dab6a7efcd', commit_message='Upload dataset', commit_description='', oid='a8dd8ffe5d1d061882bb7bd6b971c0dab6a7efcd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/harsha2946/ParaDetox-Augmented', endpoint='https://huggingface.co', repo_type='dataset', repo_id='harsha2946/ParaDetox-Augmented'), pr_revision=None, pr_num=None)

In [38]:
ds=load_dataset("harsha2946/ParaDetox-Augmented")

In [39]:
df=pd.DataFrame(ds["train"])
df.describe()

,sentence_similarity
count,39488.000000
mean,0.737618
std,0.191921
min,-0.002673
25%,0.617886
50%,0.781865
75%,0.893694
max,1.000000
